In [1]:
# импортирования библиотек
import pandas as pd
import numpy as np

## Очистка датасета


In [2]:
# чтение датасета торгов
data = pd.read_excel('raw_data.xlsx')
data.head()

,Стоимость \n(руб.) Заказчик,Сфера деятельности,Наименование публикации,Регион поставки,Поставщик,ИНН поставщика
0,0.0,[ОКПД2 29.10] Средства автотранспортные,Поставка легкового автомобиля с пробегом (мини...,Красноярский край,NaN,NaN
1,500000.0,[ОКПД2 29.32] Комплектующие и принадлежности д...,Право заключения договора на Поставка запасных...,Тверская область,Неизвестно,NaN
2,0.0,[ОКПД2 29.32] Комплектующие и принадлежности д...,Поставка рамы автомобильной ГАЗ-33086-10 для н...,Оренбургская область,NaN,NaN
3,998700.0,[ОКПД2 29.32] Комплектующие и принадлежности д...,Поставка запасных частей для грузовых автомоби...,Воронежская область,Неизвестно,NaN
4,1000000.0,[ОКПД2 27.32] Провода и кабели электронные и э...,Право заключения договора на Поставка запасных...,Москва,Неизвестно,NaN


In [3]:
# обьем датасета
data.shape

(349869, 6)

In [4]:
# исключения из датасета торгов, в которых неизвестен поставщик
arr = ['Неизвестно', 'Несколько поставщиков', 'неизвестно']
data = data.query("Поставщик not in @arr")

In [5]:
# исключения из датасета торгов, в которых неизвестна цена тендера
data = data[(data['Поставщик'].isnull()==False) &(data['Стоимость \n(руб.) Заказчик'].isnull() == False)]

In [6]:
# размер обновленного датасета
data.shape

(29840, 6)

## Преобразования данных торгов

In [7]:
# удаление записей, в которых 2 раза присутствует название региона
fixed_region = data[data['Регион поставки'].str.contains(';') == True]['Регион поставки'].apply(lambda x:x.split(';')[0])
data.loc[data['Регион поставки'].str.contains(';')==True , 'Регион поставки'] = fixed_region

In [8]:
# выбор поставщиков, у которых больше 10 успешных торгов
suppliers = pd.Series(data['ИНН поставщика'].value_counts())
prep = list(suppliers[suppliers > 9].index)
data = data[data['ИНН поставщика'].isin(prep)]

In [9]:
# размер обновленного датасета
data.shape

(17980, 6)

In [10]:
# конвертирование категориальной переменной 'Регион поставки' в числовые значения
from sklearn import preprocessing 
region_encoder = preprocessing.LabelEncoder() 
region_encoder.fit(data['Регион поставки'])

LabelEncoder()

In [11]:
# удаления дубликатов строк в данных
data.drop_duplicates(inplace = True)

In [12]:
data

,Стоимость \n(руб.) Заказчик,Сфера деятельности,Наименование публикации,Регион поставки,Поставщик,ИНН поставщика
94,1.760000e+00,[ОКПД2 29.32] Комплектующие и принадлежности д...,Приобретение запасных частей на автомобильную ...,Ростовская область,Лунин Сергей Михайлович,6.165090e+11
96,5.860000e+00,[ОКПД2 29.32] Комплектующие и принадлежности д...,Приобретение запасных частей на автомобильную ...,Ростовская область,Лунин Сергей Михайлович,6.165090e+11
111,2.593000e+01,[ОКПД2 29.32] Комплектующие и принадлежности д...,Поставка запасных частей для автомобилей КАМАЗ...,Алтайский край,"ООО ""АВТОПАРТНЕР""",2.225183e+09
137,4.475000e+01,[ОКПД2 29.32] Комплектующие и принадлежности д...,Поставка запасных частей для транспортных сред...,Новгородская область,"ООО ""БАЛТКАМ""",7.826019e+09
244,2.800000e+02,[ОКПД2 29.32] Комплектующие и принадлежности д...,Поставка запасных частей и комплектующих для а...,Самарская область,"ООО ""ГУДЗОН""",6.314047e+09
...,...,...,...,...,...,...
194642,1.143360e+10,"[ОКПД2 29.10] Средства автотранспортные, [КТРУ...",Аукцион в электронной форме на право заключени...,Москва,"ООО ""МЕРКАТОР ХОЛДИНГ""",7.709402e+09
194643,1.146667e+10,"[ОКПД2 29.10] Средства автотранспортные, [ОКПД...",Аукцион в электронной форме на право заключени...,Москва,"ООО ""МЕРКАТОР ХОЛДИНГ""",7.709402e+09
194679,1.000000e+05,[ОКПД2 29.32] Комплектующие и принадлежности д...,"19/Поставка запасных частей, жидкостей и масел...",Вологодская область,Сазанов Анатолий Николаевич,3.525000e+11
206444,9.900000e+06,[ОКПД2 29.10] Средства автотранспортные,Выбор поставщика автомобилей скорой медицинско...,Москва,"ООО ""СОГРАНДИС""",5.256120e+09


In [13]:
# кодирования региона поставки
region_res = region_encoder.fit_transform(data['Регион поставки']) 

region_encoder.classes_ = region_encoder.classes_[:-1]
region_encoder.classes_

array(['Алтайский край', 'Амурская область', 'Архангельская область',
       'Астраханская область', 'Байконур', 'Белгородская область',
       'Брянская область', 'Владимирская область',
       'Волгоградская область', 'Вологодская область',
       'Воронежская область', 'Донецкая Народная Республика',
       'Еврейская автономная область', 'Забайкальский край',
       'Запорожская область', 'Ивановская область', 'Иркутская область',
       'Кабардино-Балкарская Республика', 'Калининградская область',
       'Калужская область', 'Камчатский край',
       'Карачаево-Черкесская Республика', 'Кемеровская область - Кузбасс',
       'Кировская область', 'Костромская область', 'Краснодарский край',
       'Красноярский край', 'Курганская область', 'Курская область',
       'Ленинградская область', 'Липецкая область',
       'Луганская Народная Республика', 'Магаданская область', 'Москва',
       'Московская область', 'Мурманская область',
       'Ненецкий автономный округ (Архангельская обл

In [14]:
# функция для извлечения кода ОКПД2
def get_okpd_code(s):
    try:
        arr = s.split()
        first = arr[0]
        code = arr[1][:-1]
        if first == '[ОКПД2' and code.split('.')[0] == '29':
            return code
    except:
        return None

In [15]:
# количество тендеров по окпд2
data['Сфера деятельности'].apply(lambda x: get_okpd_code(x)).value_counts()

Сфера деятельности
29.10    8449
29.32    6056
29.20     918
29.31     913
29.3+      26
Name: count, dtype: int64

In [16]:
# функция для извлечение первой цифры после точки
def level_1(arr):
    if len(arr) == 1:
        return int(arr[0].split(".")[1][0])
    else:
        res = [int(i.split(".")[1][0]) for i in arr]
        return list(set(res))

In [17]:
# функция для извлечение второй цифры после точки
def level_2(arr):
    if len(arr) == 1:
        elem = arr[0].split(".")[1][1]
        if elem == '1' or elem == '2':
            return int(elem)
        else:
            return 0
    else:
        res = []
        for i in arr:
            if i.split(".")[1][1] == '1' or i.split(".")[1][1] == '2':
                res.append(int(i.split(".")[1][1]))
        return list(set(res))

In [18]:
# конвертирование столбца 'Сфера деятельности' в числовое значение через метод One Hot Encoding
level_1_1 = []
level_1_2 = []
level_1_3 = []
level_2_1 = []
level_2_2 = []
product = list(data['Сфера деятельности'])
for i in product:
    prep =  i.split(',')
    arr1 = []
    for j in range(len(prep)):
        if get_okpd_code(prep[j]) != None:
            arr1.append(get_okpd_code(prep[j]))
    res1 = level_1(arr1)
    if type(res1) == list:
        if 1 in res1:
            level_1_1.append(1)
        else:
            level_1_1.append(0)
        if 2 in res1:
            level_1_2.append(1)
        else:
            level_1_2.append(0)
        if 3 in res1:
            level_1_3.append(1)
        else:
            level_1_3.append(0)
    else:
        if res1 == 1:
            level_1_1.append(1)
        else:
            level_1_1.append(0)
        if res1 == 2:
            level_1_2.append(1)
        else:
            level_1_2.append(0)
        if res1 == 3:
            level_1_3.append(1)
        else:
            level_1_3.append(0)
    
    res2 = level_2(arr1)
    if type(res2) == list:
        if 1 in res2:
            level_2_1.append(1)
        else:
            level_2_1.append(0)
        if 2 in res2:
            level_2_2.append(1)
        else:
            level_2_2.append(0)
    else:
        if res2 == 1:
            level_2_1.append(1)
        else:
            level_2_1.append(0)
        if res2 == 2:
            level_2_2.append(1)
        else:
            level_2_2.append(0)

In [20]:
# конвертирования описания тендера в 20d вектор
from gensim.models.doc2vec import Doc2Vec,\
    TaggedDocument
from nltk.tokenize import word_tokenize
 

info = list(data['Наименование публикации'])
 
tagged_data = [TaggedDocument(words=word_tokenize(doc.lower()),
                              tags=[str(i)]) for i,
               doc in enumerate(info)]
# обучения модели  
model = Doc2Vec(vector_size=20,
                min_count=2, epochs=50)
model.build_vocab(tagged_data)
model.train(tagged_data,
            total_examples=model.corpus_count,
            epochs=model.epochs)
 
# взятия векторов 
document_vectors = [model.infer_vector(
    word_tokenize(doc.lower())) for doc in info]
 


In [21]:
# создание датасета с данными для обучения модели
df = pd.DataFrame()
df['cost'] = data['Стоимость \n(руб.) Заказчик']
df['level_1_1'] = level_1_1
df['level_1_2'] = level_1_2
df['level_1_3'] = level_1_3
df['level_2_1'] = level_2_1
df['level_2_2'] = level_2_2
df['region'] = list(region_res)

In [22]:
# фунция для извлечения столбца из вектора
def get_column(data, i):
    res = []
    for row in range(len(data)):
        res.append(data[row][i])
    return res

In [23]:
# создания столбцов векторов описания тендера
for i in range(20):
    df[f"title_vector_{i}"] = get_column(document_vectors, i)

In [24]:
# преобразования столбца поставщика
supplier_encoder = preprocessing.LabelEncoder() 
df['supplier_id'] = supplier_encoder.fit_transform(data['ИНН поставщика'])

In [25]:
# конвертированная таблица
df

,cost,level_1_1,level_1_2,level_1_3,level_2_1,level_2_2,region,title_vector_0,title_vector_1,title_vector_2,...,title_vector_11,title_vector_12,title_vector_13,title_vector_14,title_vector_15,title_vector_16,title_vector_17,title_vector_18,title_vector_19,supplier_id
94,1.760000e+00,0,0,1,0,1,64,-0.118420,-0.189031,0.262658,...,0.347078,0.344118,-0.099853,0.188831,-0.151217,0.158092,0.151918,-0.114262,-0.048184,571
96,5.860000e+00,0,0,1,0,1,64,-0.157906,-0.189147,0.321396,...,0.300885,0.375428,-0.011600,0.225350,-0.144752,0.143819,0.151911,-0.078590,-0.088649,571
111,2.593000e+01,0,0,1,0,1,0,0.075686,-0.028516,0.507158,...,0.176968,0.230084,-0.205547,0.419286,-0.110185,0.296528,0.146566,-0.208166,-0.017537,58
137,4.475000e+01,0,0,1,0,1,38,0.003408,-0.199682,0.260283,...,0.067205,0.330786,-0.014351,0.242984,-0.312769,0.134266,0.120170,-0.114111,-0.408749,474
244,2.800000e+02,0,0,1,0,1,66,0.233447,-0.048200,0.261783,...,-0.091009,0.611437,0.062231,-0.131013,-0.097989,0.409009,0.143789,-0.127682,-0.452190,312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194642,1.143360e+10,1,0,0,0,0,33,-0.141599,-0.246891,0.897689,...,0.138046,0.833713,0.127018,0.221907,-0.279877,0.332753,0.112803,-0.262441,-0.515463,412
194643,1.146667e+10,1,1,0,0,0,33,-0.227054,0.331933,-0.082686,...,0.018623,0.521244,0.286765,0.501515,-0.064570,0.131470,-0.049947,-0.354320,-0.166221,412
194679,1.000000e+05,0,0,1,0,1,9,0.237124,0.104846,0.097278,...,0.160853,0.286070,-0.007834,0.127547,-0.054333,0.662428,-0.172653,0.296296,-0.228132,536
206444,9.900000e+06,1,0,0,0,0,33,-0.124432,-0.181541,0.700490,...,0.247447,0.317035,0.283771,-0.205729,0.030734,0.217149,0.059138,-0.111444,-0.443075,205


## Создания таблицы поставщиков

In [26]:
# таблица средних значений векторов описания по каждому поставщику
supplier_df = df.groupby('supplier_id').agg({
    'title_vector_0':'mean',
    'title_vector_1':'mean',
    'title_vector_2':'mean',
    'title_vector_3':'mean',
    'title_vector_4':'mean',
    'title_vector_5':'mean',
    'title_vector_6':'mean',
    'title_vector_7':'mean',
    'title_vector_8':'mean',
    'title_vector_9':'mean',
    'title_vector_10':'mean',
    'title_vector_11':'mean',
    'title_vector_12':'mean',
    'title_vector_13':'mean',
    'title_vector_14':'mean',
    'title_vector_15':'mean',
    'title_vector_16':'mean',
    'title_vector_17':'mean',
    'title_vector_18':'mean',
    'title_vector_19':'mean',
    'cost':'mean'
}).reset_index()
supplier_df.head()

,supplier_id,title_vector_0,title_vector_1,title_vector_2,title_vector_3,title_vector_4,title_vector_5,title_vector_6,title_vector_7,title_vector_8,...,title_vector_11,title_vector_12,title_vector_13,title_vector_14,title_vector_15,title_vector_16,title_vector_17,title_vector_18,title_vector_19,cost
0,0,0.023195,-0.103914,0.278693,0.146083,0.070353,-0.157877,-0.307416,0.267873,-0.200302,...,-0.101981,0.246126,-0.055305,0.008130,-0.182204,0.201294,0.004667,-0.220501,-0.062232,1.849718e+06
1,1,0.001119,-0.159416,0.276630,0.097316,-0.043908,-0.170518,-0.259088,0.275022,-0.227471,...,-0.130238,0.252638,-0.088709,-0.010889,-0.171212,0.230248,0.086108,-0.277934,-0.096766,2.671392e+06
2,2,0.181170,-0.050389,0.315383,0.217944,0.096868,-0.200409,-0.375403,0.453901,-0.141410,...,0.070131,0.438787,0.057083,0.136530,-0.114893,0.199631,0.194889,-0.290491,-0.298974,1.321690e+07
3,3,-0.022728,0.001928,0.302067,0.158737,0.043300,-0.060248,-0.387567,0.296439,-0.077929,...,0.056432,0.191303,-0.112152,-0.072129,-0.096242,0.257147,0.035517,-0.212910,-0.071833,1.488335e+06
4,4,-0.111300,-0.129888,0.480927,0.305637,-0.031326,-0.126440,-0.319049,0.212701,-0.048551,...,0.004041,0.463754,0.273999,0.313831,0.041117,0.062236,0.061303,-0.242571,-0.212145,1.016041e+06


In [27]:
df['supplier_id'].value_counts()

supplier_id
135    336
316    282
189    278
232    258
237    198
      ... 
491      6
511      5
312      5
512      5
471      3
Name: count, Length: 620, dtype: int64

In [28]:
# агрегация кода окпд2 по каждому поставщику
arr = []
for i in df.index:
    level = str(df['level_1_1'].loc[i]) + str(df['level_1_2'].loc[i]) + str(df['level_1_3'].loc[i]) + str(df['level_2_1'].loc[i]) + str(df['level_2_2'].loc[i])
    
    res = [df['supplier_id'].loc[i], level]
    arr.append(res)
sup_level = pd.DataFrame(arr, columns = ['supplier_id', 'level'])
                                                                                                               

In [29]:
for i in sup_level['level'].unique():
    col = f"level_{i}"
    sup_level[col] = sup_level['level'].apply(lambda x:int(x == i))

In [30]:
sup_level = sup_level.groupby('supplier_id').sum().reset_index()

In [31]:
# обьеденение данных
supplier_df = supplier_df.merge(sup_level,on = 'supplier_id')

In [32]:
supplier_df

,supplier_id,title_vector_0,title_vector_1,title_vector_2,title_vector_3,title_vector_4,title_vector_5,title_vector_6,title_vector_7,title_vector_8,...,level_00101,level_00110,level_00100,level_00111,level_01000,level_10000,level_01101,level_10100,level_01111,level_11000
0,0,0.023195,-0.103914,0.278693,0.146083,0.070353,-0.157877,-0.307416,0.267873,-0.200302,...,0,0,0,0,0,16,0,0,0,0
1,1,0.001119,-0.159416,0.276630,0.097316,-0.043908,-0.170518,-0.259088,0.275022,-0.227471,...,0,0,0,0,0,27,0,0,0,0
2,2,0.181170,-0.050389,0.315383,0.217944,0.096868,-0.200409,-0.375403,0.453901,-0.141410,...,0,0,0,0,0,19,0,0,0,0
3,3,-0.022728,0.001928,0.302067,0.158737,0.043300,-0.060248,-0.387567,0.296439,-0.077929,...,0,0,0,0,0,10,0,0,0,0
4,4,-0.111300,-0.129888,0.480927,0.305637,-0.031326,-0.126440,-0.319049,0.212701,-0.048551,...,0,0,0,0,16,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,615,0.077897,-0.023876,0.336602,0.058051,0.082979,-0.204591,-0.192948,0.150011,-0.085990,...,34,3,0,3,0,0,0,0,0,0
616,616,-0.000080,-0.345090,0.510020,0.052198,-0.132928,-0.324146,-0.372589,0.272896,0.145054,...,0,0,0,0,12,0,0,0,0,0
617,617,0.044505,-0.098369,0.353412,0.016911,0.101062,-0.299485,-0.317866,0.200419,-0.090994,...,12,0,0,2,0,0,0,0,0,0
618,618,0.008532,-0.024701,0.328966,0.094142,0.016013,-0.093741,-0.384044,0.270822,-0.185002,...,0,0,0,0,0,42,0,0,0,0


In [33]:
# аггрегация данных региона поставки поставщиков

def create_region_frequency_encoding(df):

    region_counts = df.groupby(['supplier_id', 'region']).size().reset_index(name='region_count')

    total_regions_per_supplier = region_counts.groupby('supplier_id')['region_count'].sum().reset_index(name='total_regions')

    region_frequencies_df = pd.merge(region_counts, total_regions_per_supplier, on='supplier_id')

    region_frequencies_df['region_frequency'] = region_frequencies_df['region_count'] / region_frequencies_df['total_regions']

    supplier_region_features = region_frequencies_df.pivot_table(
        index='supplier_id', columns='region', values='region_frequency', fill_value=0
    )

    supplier_region_features.columns.name = None
    supplier_region_features = supplier_region_features.reset_index()


    return supplier_region_features



supplier_region_features_df = create_region_frequency_encoding(df)

In [34]:
supplier_region_features_df

,supplier_id,0,1,2,3,4,5,6,7,8,...,81,82,83,84,85,86,87,88,89,90
0,0,0.0,0.000,0.00000,0.000,0.0,0.000000,0.0,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000
1,1,0.0,0.000,0.00000,0.000,0.0,0.000000,0.0,0.00000,0.037037,...,0.000000,0.000000,0.037037,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000
2,2,0.0,0.000,0.00000,0.000,0.0,0.052632,0.0,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000
3,3,0.0,0.000,0.00000,0.000,0.0,0.000000,0.0,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000
4,4,0.0,0.000,0.00000,0.000,0.0,0.000000,0.0,0.00000,0.062500,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,615,0.0,0.025,0.02500,0.025,0.0,0.000000,0.0,0.00000,0.025000,...,0.025000,0.050000,0.000000,0.0,0.0,0.0,0.05,0.0,0.025000,0.000000
616,616,0.0,0.000,0.00000,0.000,0.0,0.000000,0.0,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.083333,0.000000
617,617,0.0,0.000,0.00000,0.000,0.0,0.000000,0.0,0.00000,0.000000,...,0.785714,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.142857
618,618,0.0,0.000,0.02381,0.000,0.0,0.000000,0.0,0.02381,0.000000,...,0.000000,0.071429,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000


In [35]:
# обьеденение данных
supplier_df = supplier_df.merge(supplier_region_features_df,on = 'supplier_id')

In [36]:
supplier_df

,supplier_id,title_vector_0,title_vector_1,title_vector_2,title_vector_3,title_vector_4,title_vector_5,title_vector_6,title_vector_7,title_vector_8,...,81,82,83,84,85,86,87,88,89,90
0,0,0.023195,-0.103914,0.278693,0.146083,0.070353,-0.157877,-0.307416,0.267873,-0.200302,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000
1,1,0.001119,-0.159416,0.276630,0.097316,-0.043908,-0.170518,-0.259088,0.275022,-0.227471,...,0.000000,0.000000,0.037037,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000
2,2,0.181170,-0.050389,0.315383,0.217944,0.096868,-0.200409,-0.375403,0.453901,-0.141410,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000
3,3,-0.022728,0.001928,0.302067,0.158737,0.043300,-0.060248,-0.387567,0.296439,-0.077929,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000
4,4,-0.111300,-0.129888,0.480927,0.305637,-0.031326,-0.126440,-0.319049,0.212701,-0.048551,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,615,0.077897,-0.023876,0.336602,0.058051,0.082979,-0.204591,-0.192948,0.150011,-0.085990,...,0.025000,0.050000,0.000000,0.0,0.0,0.0,0.05,0.0,0.025000,0.000000
616,616,-0.000080,-0.345090,0.510020,0.052198,-0.132928,-0.324146,-0.372589,0.272896,0.145054,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.083333,0.000000
617,617,0.044505,-0.098369,0.353412,0.016911,0.101062,-0.299485,-0.317866,0.200419,-0.090994,...,0.785714,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.142857
618,618,0.008532,-0.024701,0.328966,0.094142,0.016013,-0.093741,-0.384044,0.270822,-0.185002,...,0.000000,0.071429,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000


In [37]:
# переименования столбцов датасета
supplier_df.rename({
    'title_vector_0':'supplier_title_vector_0',
    'title_vector_1':'supplier_title_vector_1',
    'title_vector_2':'supplier_title_vector_2',
    'title_vector_3':'supplier_title_vector_3',
    'title_vector_4':'supplier_title_vector_4',
    'title_vector_5':'supplier_title_vector_5',
    'title_vector_6':'supplier_title_vector_6',
    'title_vector_7':'supplier_title_vector_7',
    'title_vector_8':'supplier_title_vector_8',
    'title_vector_9':'supplier_title_vector_9',
    'title_vector_10':'supplier_title_vector_10',
    'title_vector_11':'supplier_title_vector_11',
    'title_vector_12':'supplier_title_vector_12',
    'title_vector_13':'supplier_title_vector_13',
    'title_vector_14':'supplier_title_vector_14',
    'title_vector_15':'supplier_title_vector_15',
    'title_vector_16':'supplier_title_vector_16',
    'title_vector_17':'supplier_title_vector_17',
    'title_vector_18':'supplier_title_vector_18',
    'title_vector_19':'supplier_title_vector_19',
    'cost':'supplier_cost'},axis = 1,inplace = True)

In [38]:
df.index = range(len(df))

In [39]:
df['tender_id'] = np.array(range(len(df)))

In [40]:
supplier_df

,supplier_id,supplier_title_vector_0,supplier_title_vector_1,supplier_title_vector_2,supplier_title_vector_3,supplier_title_vector_4,supplier_title_vector_5,supplier_title_vector_6,supplier_title_vector_7,supplier_title_vector_8,...,81,82,83,84,85,86,87,88,89,90
0,0,0.023195,-0.103914,0.278693,0.146083,0.070353,-0.157877,-0.307416,0.267873,-0.200302,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000
1,1,0.001119,-0.159416,0.276630,0.097316,-0.043908,-0.170518,-0.259088,0.275022,-0.227471,...,0.000000,0.000000,0.037037,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000
2,2,0.181170,-0.050389,0.315383,0.217944,0.096868,-0.200409,-0.375403,0.453901,-0.141410,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000
3,3,-0.022728,0.001928,0.302067,0.158737,0.043300,-0.060248,-0.387567,0.296439,-0.077929,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000
4,4,-0.111300,-0.129888,0.480927,0.305637,-0.031326,-0.126440,-0.319049,0.212701,-0.048551,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,615,0.077897,-0.023876,0.336602,0.058051,0.082979,-0.204591,-0.192948,0.150011,-0.085990,...,0.025000,0.050000,0.000000,0.0,0.0,0.0,0.05,0.0,0.025000,0.000000
616,616,-0.000080,-0.345090,0.510020,0.052198,-0.132928,-0.324146,-0.372589,0.272896,0.145054,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.083333,0.000000
617,617,0.044505,-0.098369,0.353412,0.016911,0.101062,-0.299485,-0.317866,0.200419,-0.090994,...,0.785714,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.142857
618,618,0.008532,-0.024701,0.328966,0.094142,0.016013,-0.093741,-0.384044,0.270822,-0.185002,...,0.000000,0.071429,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000


In [45]:
# сохранение данных поставщиков в csv файл
supplier_df.to_csv('supplier_data.csv')

In [47]:
# сохранение данных тендера в csv файл
df.to_csv('data.csv')

In [55]:
# сохранение модели описания тендера
import pickle
filename = 'description_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [42]:
# сохранение кодировки поставщиков в словарь
suppliers_encoder_json = {}
for index, value in enumerate(supplier_encoder.classes_):
    suppliers_encoder_json[int(value)] = index

region_encoder_json = {}
for index, value in enumerate(region_encoder.classes_):
    region_encoder_json[value] = index


In [43]:
#сохранение словарей в json

import json
with open("supplier_encoder.json", "w") as outfile: 
    json.dump(suppliers_encoder_json, outfile)
    
with open("region_encoder.json", "w") as outfile: 
    json.dump(region_encoder_json, outfile)